In [2]:
import pandas as pd

chunk_iter = pd.read_csv('crunchbase-investments.csv', encoding = 'ISO-8859-1', chunksize=5000)
total_memory_usage = 0

#Each column's missing value counts
missing = []
for chunk in chunk_iter:
    missing.append(chunk.isnull().sum())

missing = pd.concat(missing)    
missing = missing.groupby(missing.index).sum()
missing.sort_values()


company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

Each column's memory footprint

In [3]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', encoding = 'ISO-8859-1', chunksize=5000)
counter = 0
memory = pd.Series()
for chunk in chunk_iter:
    if counter == 0:
        memory = chunk.memory_usage(deep=True) / (1024**2)
    else:
        memory += chunk.memory_usage(deep=True) / (1024**2)
    counter += 1

# Drop memory footprint calculation for the index.
memory = memory.drop('Index')
memory

company_permalink         3.869808
company_name              3.424955
company_category_code     3.262619
company_country_code      3.025223
company_state_code        2.962161
company_region            3.253541
company_city              3.343512
investor_permalink        4.749821
investor_name             3.734270
investor_category_code    0.593590
investor_country_code     2.524654
investor_state_code       2.361876
investor_region           3.238946
investor_city             2.751430
funding_round_type        3.252704
funded_at                 3.378091
funded_month              3.226837
funded_quarter            3.226837
funded_year               0.403366
raised_amount_usd         0.403366
dtype: float64

The total memory footprint of all of the chunks combined

In [4]:
print(str(round(memory.sum(),2)) + " Megabytes")

56.99 Megabytes


Which column(s) we can drop because they aren't useful for analysis

In [5]:
# Drop columns representing URL's or containing way too many missing values (>90% missing)
drop_cols = ['investor_permalink', 'company_permalink', 'investor_category_code']
keep_cols = list(chunk.columns.drop(drop_cols))


Identify the types for each column.

In [6]:
df = pd.read_csv('crunchbase-investments.csv', encoding = 'ISO-8859-1', usecols=keep_cols)
df.dtypes

company_name              object
company_category_code     object
company_country_code      object
company_state_code        object
company_region            object
company_city              object
investor_name             object
investor_country_code     object
investor_state_code       object
investor_region           object
investor_city             object
funding_round_type        object
funded_at                 object
funded_month              object
funded_quarter            object
funded_year              float64
raised_amount_usd        float64
dtype: object

Identify the numeric columns we can represent using more space efficient types.

In [15]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numericsdf = df.select_dtypes(include=numerics)
numericsdf

,funded_year,raised_amount_usd
0,2012.0,2000000.0
1,2012.0,20000.0
2,2012.0,20000.0
3,2012.0,20000.0
4,2011.0,20000.0
5,2012.0,20000.0
6,2012.0,20000.0
7,2012.0,70000.0
8,2010.0,75000.0
9,2011.0,100000.0


In [8]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)
unique_value_counts = []
for chunk in chunk_iter:
    unique_value_counts.append(chunk.select_dtypes(include=['object']).columns.value_counts())
print(unique_value_counts[-1])

funded_month             1
company_city             1
company_region           1
company_country_code     1
company_state_code       1
company_category_code    1
funded_at                1
funding_round_type       1
company_name             1
investor_region          1
investor_name            1
funded_quarter           1
dtype: int64


Create and connect to SQLite Server

In [24]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')


Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.

In [22]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
for chunk in chunk_iter:
    chunk.to_sql("fundraising", conn, if_exists='append', index=False)

Query the table and make sure the data types match up to what you had in mind for each column.

In [23]:
results_df = pd.read_sql('PRAGMA table_info(fundraising)',conn)
results_df

,cid,name,type,notnull,dflt_value,pk
0,0,company_permalink,TEXT,0,None,0
1,1,company_name,TEXT,0,None,0
2,2,company_category_code,TEXT,0,None,0
3,3,company_country_code,TEXT,0,None,0
4,4,company_state_code,TEXT,0,None,0
5,5,company_region,TEXT,0,None,0
6,6,company_city,TEXT,0,None,0
7,7,investor_permalink,TEXT,0,None,0
8,8,investor_name,TEXT,0,None,0
9,9,investor_category_code,TEXT,0,None,0


Use the !wc IPython command to return the file size of the database.

In [20]:
!wc -c 'crunchbase-investments.csv'

10339663 crunchbase-investments.csv


### Use the pandas SQLite workflow to answer the following questions:

What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.

Which category of company attracted the most investments?

In [31]:
most_investments = pd.read_sql('SELECT company_category_code, COUNT(company_category_code) as count FROM fundraising GROUP BY company_category_code ORDER BY count DESC',conn)
most_investments.head(n=1)

,company_category_code,count
0,software,28972


Which investor contributed the most money (across all startups)?

In [44]:
biggest_investor = pd.read_sql('SELECT investor_name,company_name, COUNT(company_name) as count FROM fundraising GROUP BY investor_name ORDER BY count DESC',conn)
biggest_investor.iloc[0]

investor_name    New Enterprise Associates
company_name                    Mattermark
count                                 1780
Name: 0, dtype: object

Which investors contributed the most money per startup?

In [ ]:
investors_per_startup = pd.read_sql('SELECT investor_name, COUNT(funding_round_type) as count FROM fundraising GROUP BY funding_round_type ORDER BY count DESC',conn)
investors_per_startup.loc[0]

Which funding round was the most popular? Which was the least popular?

In [39]:
most_popular_funding_round = pd.read_sql('SELECT funding_round_type, COUNT(funding_round_type) as count FROM fundraising GROUP BY funding_round_type ORDER BY count DESC',conn)
most_popular_funding_round.loc[0]

funding_round_type    series-a
count                    55752
Name: 0, dtype: object

Which was the least popular?

In [38]:
least_popular_funding_round = pd.read_sql('SELECT funding_round_type, COUNT(funding_round_type) as count FROM fundraising GROUP BY funding_round_type ORDER BY count',conn)
least_popular_funding_round.loc[1]

funding_round_type    crowdfunding
count                           20
Name: 1, dtype: object